In [1]:
import ROOT
import numpy as np
import toml
from dataclasses import dataclass

%jsroot on

Welcome to JupyROOT 6.28/00


# Classes

Using a dataclass to store config information from toml file

In [2]:
@dataclass(frozen=True)
class Config:
    stage: str
    tree_name: str
    numch: int
    nuclei: str
    fp_length: int
    pu_flag: int
    fo_flag: int
    
    hTOF_mus_down: int
    hTOF_mus_up: int
    hTOF_mus_Nbins: int

    hTOF_ns_down: int
    hTOF_ns_up: int
    hTOF_ns_Nbins: int

    hPulseHeight_down: int
    hPulseHeight_up: int
    hPulseHeight_Nbins: int

    hEn_cuts_on_hEgam: list[str]
    hEgam_cuts_on_hEn: list[str]
    
    @staticmethod
    def new(file_name: str) -> "Config":
        """Reads the stage0 config file and returns a Config object"""
        config = toml.load(file_name)
        
        return Config(**config['general'], 
                      **config['flags'], 
                      **config['binning'], 
                      **config['cuts'])

In [3]:
config = Config.new("stage0_config.toml")
# config.tree_name

In [4]:
# hist = ROOT.TH1D("test","Test", 10, -3, 3)
# hist.FillRandom("gaus",10000)
# hist.Draw()
# ROOT.gROOT.GetListOfCanvases().Draw()

In [5]:
def tof_1mus_to_En(TOF: any, L: float) -> any:
    """Converts TOF in mu s to neutron energy in eV using the length from moderator to target L in m"""

    return ((72.3 * L)/ TOF)**2

def tof_10ns_to_En(TOF: any, L: float) -> any:
    """Converts TOF in units of 10 ns to neutron energy in eV using the length from moderator to target L in m"""

    return ((72.3 * L)/ (TOF/100))**2

def tof_1ns_to_En(TOF: any, L: float) -> any:
    """Converts TOF in units of ns to neutron energy in eV using the length from moderator to target L in m"""

    return ((72.3 * L)/ (TOF*10))**2

In [6]:
# TOF_10ns_to_En(48000,21.5)

In [7]:
Nbins = 100000
down = 0
up =100000

# xbins_tof = np.linspace(1, 100000+2, 100000+1)
xbins_tof = np.linspace(up/Nbins, up+up/Nbins, Nbins+1)
print(xbins_tof)
xbins_En=tof_10ns_to_En(xbins_tof,21.5)
xbins_En=np.sort(xbins_En)
print(xbins_En)

[1.00000e+00 2.00000e+00 3.00000e+00 ... 9.99990e+04 1.00000e+05
 1.00001e+05]
[2.41626648e+00 2.41631480e+00 2.41636313e+00 ... 2.68479422e+09
 6.04078701e+09 2.41631480e+10]


In [8]:
# xbins = np.linspace(-3,3,10+1)
# print(xbins)
# hist1 = ROOT.TH1D("test1","Test", 10, xbins)
# hist1.FillRandom("gaus",10000)
# hist1.Draw()
# ROOT.gROOT.GetListOfCanvases().Draw()

In [9]:
# ROOT.EnableImplicitMT()

# tree_name = "rawTree"
# # file_name = "/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_001[4-9]*"
# file_name = "rawroot/rawroot*"

# df = ROOT.RDataFrame(tree_name, file_name)
# # print(df.GetColumnNames())
# # print(df.GetColumnType("tof"))

# df = df.Define("En","pow((72.3*21.5/(tof/100.0)),2)").Define("tof_mu","tof/100.0").Filter("detector == 1")
# df1 = df.Filter("detector == 1")
# df2 = df.Filter("detector == 2")


# h = df.Histo1D(hist_model,"En")
# h1 = df.Histo1D(("hEn1","hEn1;E_{n } [eV];counts",10000,0,100),"En")
# h1.SetLineColor(ROOT.kRed)
# # h2 = df2.Histo1D(("hEn2","hEn2;E_{n } [eV];counts",10000,0,100),"En")
# h2 = df.Histo1D(("tof","htof;10 ns;counts",100000,0,100000),"tof")
# # print(df.GetDefinedColumnNames())
# # print(df.GetColumnNames())

# # h.ResetStats()
# h.Draw()
# h1.Draw("hist same")
# # d1 = df.Display("TOF", 128)
# # h2.Draw()

# print(df.GetNRuns())
# # df.Describe().Print()
# # df.Filter("detector == 1").Display("tof").Print()
# # d1.Print()
# # print(df1.GetNRuns())
# # ROOT.gROOT.GetListOfCanvases().Draw()

In [10]:
# h2.Draw()
# # ROOT.gROOT.GetListOfCanvases().Draw()
# print(df1.GetNRuns())

In [11]:
# ROOT.EnableImplicitMT()

numch = 32

bit_puflag=32768
bit_foflag=65536

calib = [1.1]*32

tree_name = "rawTree"
# file_name = "/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_001[4-9]*"
file_name = "rawroot/rawroot*"

df = ROOT.RDataFrame(tree_name, file_name)
df.Display("").Print()
df.Describe().Print()
# print(df.Max("nTrigger").GetValue())

# df = df.Define("tof_ns","tof*10.0").Define("tof_mus","tof_ns*0.001").Define("En","pow((72.3*21.5/(tof/100.0)),2)")
df = (df.Define("tof_ns","tof*10.0")
      .Define("tof_mus","tof_ns/1000.0+0.0000001")
      .Define("En","pow((72.3*21.5/(tof_ns/1000.0)),2)+0.00001"))



# df = df.Range(1000000)


# df_pu = df.Filter(f"Flags == {bit_puflag}")  
# df = df.Filter(f"Flags != {bit_puflag}")
# print(df.Count().GetValue())


df_ch = []
df_ch_pu = []


hTOF_mus = []
hTOF_ns = []
hEn = []
hPulseHeight = []
hEgam = []


hTOF_pu = []
hEn_pu = []
hPulseHeight_pu = []
hEgam_pu = []


hEn_model = []
hTOF_mus_model = []
hTOF_ns_model = []
hPulseHeight_model = []
hEgam_model = []


for ch in range(numch):
    hTOF_mus_model.append(ROOT.RDF.TH1DModel(f"hTOF_d{ch}",
                                             f"hTOF_d{ch};TOF [#mus];counts",
                                             config.hTOF_mus_Nbins, 
                                             config.hTOF_mus_down, 
                                             config.hTOF_mus_up))
    hTOF_ns_model.append(ROOT.RDF.TH1DModel(f"hTOF_d{ch}",
                                            f"hTOF_d{ch};TOF [ns];counts",
                                            config.hTOF_ns_Nbins, 
                                            config.hTOF_ns_down, 
                                            config.hTOF_ns_up))
    hEn_model.append(ROOT.RDF.TH1DModel(f"hEn_d{ch}",
                                        f"hEn_d{ch};En [eV];counts",
                                        len(xbins_En)-1,
                                        xbins_En))
    hPulseHeight_model.append(ROOT.RDF.TH1DModel(f"hPulseHeight_d{ch}",
                                                 f"hPulseHeight_d{ch};Pulse Height;counts",
                                                 config.hPulseHeight_Nbins, 
                                                 config.hPulseHeight_down, 
                                                 config.hPulseHeight_up))
    hEgam_model.append(ROOT.RDF.TH1DModel(f"hEgam_d{ch}",
                                          f"hEgam_d{ch};Egam [keV];counts",
                                          config.hPulseHeight_Nbins,
                                          config.hPulseHeight_down,
                                          config.hPulseHeight_up*calib[ch]))
    # hTOF_model.append(ROOT.RDF.TH1DModel(f"hTOF_d{ch}",f"hTOF_d{ch};TOF [#mus];counts",10000,0,100000))
    # hEn_model.append(ROOT.RDF.TH1DModel(f"hEn_d{ch}",f"hEn_d{ch};En [eV];counts",1000,0,100))
    # hPulseHeight_model.append(ROOT.RDF.TH1DModel(f"hPulseHeight_d{ch}",f"hPulseHeight_d{ch};Pulse Height;counts",10000,0,10000))
    # hEgam_model.append(ROOT.RDF.TH1DModel(f"hEgam_d{ch}",f"hEgam_d{ch};Egam [keV];counts",10000,0,10000*calib[ch]))


for ch in range(numch):
    
    df_ch.append(df.Filter(f"detector == {ch}").Define("Egam",f"PulseHeight*{calib[ch]}"))
    # df_ch_pu.append(df_pu.Filter(f"detector == {ch}").Define("Egam",f"PulseHeight*{calib[ch]}"))

    hTOF_mus.append(df_ch[ch].Histo1D(hTOF_mus_model[ch],"tof_mus"))
    hTOF_ns.append(df_ch[ch].Histo1D(hTOF_ns_model[ch],"tof_ns"))
    hEn.append(df_ch[ch].Histo1D(hEn_model[ch],"En"))
    hPulseHeight.append(df_ch[ch].Histo1D(hPulseHeight_model[ch],"PulseHeight"))
    hEgam.append(df_ch[ch].Histo1D(hEgam_model[ch],"Egam"))

    # hTOF_pu.append(df_ch_pu[ch].Histo1D(hTOF_model[ch],"tof_ns"))
    # hEn_pu.append(df_ch_pu[ch].Histo1D(hEn_model[ch],"En"))
    # hPulseHeight_pu.append(df_ch_pu[ch].Histo1D(hPulseHeight_model[ch],"PulseHeight"))
    # hEgam_pu.append(df_ch_pu[ch].Histo1D(hEgam_model[ch],"Egam"))


# df.Describe().Print()

+-----+------+-------+-------------+-------------+----------+----------+---------+
| Row | Coin | Flags | PulseHeight | Timestamp   | detector | nTrigger | tof     | 
+-----+------+-------+-------------+-------------+----------+----------+---------+
| 0   | 0    | 0     | 481         | 3941640000  | 20       | 0        | 394164  | 
+-----+------+-------+-------------+-------------+----------+----------+---------+
| 1   | 1    | 0     | 435         | 30796670000 | 21       | 0        | 3079667 | 
+-----+------+-------+-------------+-------------+----------+----------+---------+
| 2   | 1    | 0     | 435         | 30796670000 | 22       | 0        | 3079667 | 
+-----+------+-------+-------------+-------------+----------+----------+---------+
| 3   | 1    | 0     | 435         | 30796670000 | 23       | 0        | 3079667 | 
+-----+------+-------+-------------+-------------+----------+----------+---------+
| 4   | 0    | 0     | 1343        | 4094890000  | 19       | 0        | 409489  |

In [12]:
%%time

hTOF_mus[1].Draw()
# hEn[1].Draw()
# hPulseHeight[0].Draw()
# hEgam[1].Draw()
# hEgam[1].Draw("same")
ROOT.gROOT.GetListOfCanvases().Draw()
print(f"{df.GetNRuns()} event loop(s)")

2 event loop(s)
CPU times: user 3min 43s, sys: 1.4 s, total: 3min 44s
Wall time: 3min 44s


Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [13]:
hTOF_ns[1].Draw()
ROOT.gROOT.GetListOfCanvases().Draw()

In [14]:
hEn[1].Draw()
ROOT.gROOT.GetListOfCanvases().Draw()

In [15]:
print(tof_10ns_to_En(35135,21.5))
print(tof_10ns_to_En(35136,21.5))
print(xbins_En[64999])
print(xbins_En[64866])

19.573730239188613
19.572616085517357
19.72276470126593
19.573730239188613


In [16]:
%%cpp

printf("%.64f\n", pow((72.3*21.5/(35135/100.0)),2));
printf("%.64f\n", pow((72.3*21.5/(35136/100.0)),2));

19.5737302391886132113540952559560537338256835937500000000000000000
19.5726160855173567654219368705525994300842285156250000000000000000
